<img src="https://www.xoriant.com/cdn/ff/weqpbrtpXGjLpVQ_X-gWqsFlvjAxpv5Wv3xNW0A4vuQ/1602007254/public/2020-10/a-decisionTreesforClassification-AMachineLearningAlgorithm.jpg" width=600 height=600 />




# Decision Tree - Zoning

The goal of this script is to split a review into different sentences and then classify each of them into different predefined categories

<table>
  <tr>
    <th>Color</th>
    <th>Signification</th>
  </tr>
  <tr>
    <td>Purple</td>
    <td>Introduction</td>
  </tr>
  <tr>
    <td>Red</td>
    <td>Plot</td>
  </tr>
  <tr>
    <td>Blue</td>
    <td>General Analysis of the Play</td>
  </tr>
  <tr>
    <td>Green</td>
    <td>Visual, Auditory and Audible Details</td>
  </tr>
  <tr>
    <td>Orange</td>
    <td>Actors’ Performances</td>
  </tr>
  <tr>
    <td>Brown</td>
    <td>Remarks on the Structure of the Play</td>
  </tr>
  <tr>
    <td>Yellow</td>
    <td>Conclusion</td>
  </tr>
</table>

## Table Of Contents:

* [I - Load Packages](#1-bullet)
* [II - Load Data](#2-bullet)
* [III - Data Preprocessing](#3-bullet)
* [IV - Building a Model](#4-bullet)
* [V - Model Prediction](#5-bullet)

## I - Load Packages <a class="anchor" id="1-bullet"></a>

In [12]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics
from textblob import TextBlob
import text2emotion as te
import nltk
from gensim.summarization import keywords
from sklearn.tree import export_graphviz
 
from IPython.display import Image  
import pydotplus
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/gaetanbrison/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
pip install pydotplus

Note: you may need to restart the kernel to use updated packages.


In [ ]:
display(df_repeated[df_repeated["Intro-Purple"].isna() == True].shape)
display(df_repeated[df_repeated["Red-Plot"].isna() == True].shape)
display(df_repeated[df_repeated["Blue-General"].isna() == True].shape)
display(df_repeated[df_repeated["Orange-Perf"].isna() == True].shape)
display(df_repeated[df_repeated["Borwn-Structure"].isna() == True].shape)
display(df_repeated[df_repeated["Yellow-Conclusion"].isna() == True].shape)

## II - Load Data <a class="anchor" id="2-bullet"></a>

In [13]:
df = pd.read_excel("zoning.xlsx")
df.head(5)

,Blog,Publication Date,Reviewer,Title of the Play,Playwright,Theatre,Review with Zoning,Intro-Purple,Red-Plot,Blue-General,Green-Details,Orange-Perf,Borwn-Structure,Yellow-Conclusion,Latitude,Longitude,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Start_Review_1,End_Review_1,Start_Review_2,End_Review_2,Start_Review_3,End_Review_3,Nbr_of_Sentences,Length_Start_Count_Words_1,Length_End_Count_Words_1,Length_Start_Count_Words_2,Length_End_Count_Words_2,Length_Start_Count_Words_3,Length_End_Count_Words_3,Question_Mark_Start_1,Question_Mark_End_1,Question_Mark_Start_2,Question_Mark_End_2,Question_Mark_Start_3,Question_Mark_End_3,Exclamation_Mark_Start_1,Exclamation_Mark_End_1,Exclamation_Mark_Start_2,Exclamation_Mark_End_2,Exclamation_Mark_Start_3,Exclamation_Mark_End_3,Word_Average_Start_1,Word_Average_End_1,Word_Average_Start_2,Word_Average_End_2,Word_Average_Start_3,Word_Average_End_3
0,DAILY TELEGRAPH,09.03.10,Charles Spencer,SWEET NOTHINGS,Arthur Schnitzler,Young Vic Theatre,Sigmund Freud was a great admirer of the Austr...,Sigmund Freud was a great admirer of the Austr...,"In the first act, we watch as two Viennese po...",NaN,NaN,There are some superb performances. Tom Hughe...,NaN,With strong support from David Sibley as Chris...,51.503566,-0.101745,0,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,2,0,0,0,1,1,0,2,Story_Cat,0.217,0.691,0.092,-0.9787,-0.089583,0.531548,0.15,0.39,0.09,0.24,0.12,"[('falling', 0.1670849527972756), ('cally', 0....",['Frank More just wants to teach history'],"["" Nevertheless, Milner's rabbithole view of a...","['Frank More just wants to teach history', "" B...","["" There are some breathlessly funny sequences...","['Frank More just wants to teach history', "" B...","[' But as events spiral into chaos, a mix of s...",8,7,28,58,43,76,63,0,0,1,0,1,0,0,0,0,0,0,0,4.57,5.93,5.02,5.59,4.76,5.38
1,DAILY EXPRESS,10.03.10,Paul Callan,LOVE NEVER DIES,Andrew Lloyd,Adelphi Theatre,Love never dies has suffered a rocky journey t...,Love never dies has suffered a rocky journey t...,"Lloyd Webber, with lyrics by Glen Slater, has ...",NaN,Some of the songs are lush and romantic. ’Til ...,"Similarly the American-born Miss Boggess, maki...",NaN,This is an elegant and clever sequel to Phanto...,51.510052,-0.122881,1,0,0,0,0,0,7,0,0,1,0,0,0,1,2,1,1,0,0,0,0,1,4,1,0,3,0,2,1,2,4,Show_Cat,0.039,0.781,0.180,0.9968,0.182049,0.560692,0.05,0.30,0.18,0.32,0.15,"[('singing', 0.1669848885915849), ('lloyd', 0....",['LOVE NEVER DIES has suffered a rocky journey...,[' It is a great night out'],['LOVE NEVER DIES has suffered a rocky journey...,[' This is an elegant and clever sequel to Pha...,['LOVE NEVER DIES has suffered a rocky journey...,[' And the special effects -they include singi...,25,11,7,30,35,51,60,0,0,0,0,0,0,0,0,0,0,0,0,4.27,3.00,5.00,4.88,4.82,4.81
2,SUNDAY EXPRESS,28.03.10,Mark Shenton,THE WHITE GUARD,Andrew Upton,Lyttelton Theatre,The clock is turned back in every case this we...,The clock is turned back in every case this we...,"The sole woman, Elena (a forceful Justine Mitc...",NaN,NaN,NaN,NaN,"Written across a broad, epic canvas and given...",51.507219,-0.114367,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.060,0.809,0.131,0.9052,0.195650,0.377885,0.07,0.55,0.10,0.10,0.17,"[('davies', 0.294100664407846), ('forces', 0.2...",n.a,n.a,n.a,n.a,n.a,n.a,6,1,1,2,2,3,3,0,0,0,0,0,0,0,0,0,0,0,0,1.00,1.00,1.00,1.00,1.00,1.00
3,THE TIMES,26.03.10,Sam Marlowe,GAMBLING,Tom Holloway,Soho Theatre,Garish lights wink in fetid semidarkness rent ...,"Written across a broad, epic canvas and given...",A woman sele

In [50]:
display(df[df["Intro-Purple"].isna() == True].shape[0])
display(df[df["Red-Plot"].isna() == True].shape)
display(df[df["Blue-General"].isna() == True].shape)
display(df[df["Green-Details"].isna() == True].shape)
display(df[df["Orange-Perf"].isna() == True].shape)
display(df[df["Borwn-Structure"].isna() == True].shape)
display(df[df["Yellow-Conclusion"].isna() == True].shape)

5

(0, 91)

(18, 91)

(23, 91)

(18, 91)

(39, 91)

(6, 91)

SyntaxError: invalid syntax (<ipython-input-49-a84aaacffaee>, line 1)

In [14]:
df.shape

(50, 91)

In [15]:
df["Intro-Purple"].nunique()

45

## III - Data Preprocessing <a class="anchor" id="3-bullet"></a>

In [16]:
df_repeated = pd.concat([df]*7, ignore_index=True)
display(df_repeated.head(3))
display(df_repeated.shape)

,Blog,Publication Date,Reviewer,Title of the Play,Playwright,Theatre,Review with Zoning,Intro-Purple,Red-Plot,Blue-General,Green-Details,Orange-Perf,Borwn-Structure,Yellow-Conclusion,Latitude,Longitude,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Start_Review_1,End_Review_1,Start_Review_2,End_Review_2,Start_Review_3,End_Review_3,Nbr_of_Sentences,Length_Start_Count_Words_1,Length_End_Count_Words_1,Length_Start_Count_Words_2,Length_End_Count_Words_2,Length_Start_Count_Words_3,Length_End_Count_Words_3,Question_Mark_Start_1,Question_Mark_End_1,Question_Mark_Start_2,Question_Mark_End_2,Question_Mark_Start_3,Question_Mark_End_3,Exclamation_Mark_Start_1,Exclamation_Mark_End_1,Exclamation_Mark_Start_2,Exclamation_Mark_End_2,Exclamation_Mark_Start_3,Exclamation_Mark_End_3,Word_Average_Start_1,Word_Average_End_1,Word_Average_Start_2,Word_Average_End_2,Word_Average_Start_3,Word_Average_End_3
0,DAILY TELEGRAPH,09.03.10,Charles Spencer,SWEET NOTHINGS,Arthur Schnitzler,Young Vic Theatre,Sigmund Freud was a great admirer of the Austr...,Sigmund Freud was a great admirer of the Austr...,"In the first act, we watch as two Viennese po...",NaN,NaN,There are some superb performances. Tom Hughe...,NaN,With strong support from David Sibley as Chris...,51.503566,-0.101745,0,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,2,0,0,0,1,1,0,2,Story_Cat,0.217,0.691,0.092,-0.9787,-0.089583,0.531548,0.15,0.39,0.09,0.24,0.12,"[('falling', 0.1670849527972756), ('cally', 0....",['Frank More just wants to teach history'],"["" Nevertheless, Milner's rabbithole view of a...","['Frank More just wants to teach history', "" B...","["" There are some breathlessly funny sequences...","['Frank More just wants to teach history', "" B...","[' But as events spiral into chaos, a mix of s...",8,7,28,58,43,76,63,0,0,1,0,1,0,0,0,0,0,0,0,4.57,5.93,5.02,5.59,4.76,5.38
1,DAILY EXPRESS,10.03.10,Paul Callan,LOVE NEVER DIES,Andrew Lloyd,Adelphi Theatre,Love never dies has suffered a rocky journey t...,Love never dies has suffered a rocky journey t...,"Lloyd Webber, with lyrics by Glen Slater, has ...",NaN,Some of the songs are lush and romantic. ’Til ...,"Similarly the American-born Miss Boggess, maki...",NaN,This is an elegant and clever sequel to Phanto...,51.510052,-0.122881,1,0,0,0,0,0,7,0,0,1,0,0,0,1,2,1,1,0,0,0,0,1,4,1,0,3,0,2,1,2,4,Show_Cat,0.039,0.781,0.180,0.9968,0.182049,0.560692,0.05,0.30,0.18,0.32,0.15,"[('singing', 0.1669848885915849), ('lloyd', 0....",['LOVE NEVER DIES has suffered a rocky journey...,[' It is a great night out'],['LOVE NEVER DIES has suffered a rocky journey...,[' This is an elegant and clever sequel to Pha...,['LOVE NEVER DIES has suffered a rocky journey...,[' And the special effects -they include singi...,25,11,7,30,35,51,60,0,0,0,0,0,0,0,0,0,0,0,0,4.27,3.00,5.00,4.88,4.82,4.81
2,SUNDAY EXPRESS,28.03.10,Mark Shenton,THE WHITE GUARD,Andrew Upton,Lyttelton Theatre,The clock is turned back in every case this we...,The clock is turned back in every case this we...,"The sole woman, Elena (a forceful Justine Mitc...",NaN,NaN,NaN,NaN,"Written across a broad, epic canvas and given...",51.507219,-0.114367,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.060,0.809,0.131,0.9052,0.195650,0.377885,0.07,0.55,0.10,0.10,0.17,"[('davies', 0.294100664407846), ('forces', 0.2...",n.a,n.a,n.a,n.a,n.a,n.a,6,1,1,2,2,3,3,0,0,0,0,0,0,0,0,0,0,0,0,1.00,1.00,1.00,1.00,1.00,1.00


(350, 91)

In [51]:
df_repeated["Intro-Purple"][i]

"Somewhere in this mass of material lurks a brilliant play but, like the terrorist mastermind, it hasn't been winkled out yet."

In [17]:
df["Review with Zoning"].nunique()

50

In [18]:
list_part = []
list_target = []
for i in range(0,50):
    list_part.append(df_repeated["Intro-Purple"][i])
    list_target.append(1)

In [19]:
for i in range(50,100):
    list_part.append(df_repeated["Red-Plot"][i])
    list_target.append(2)

In [20]:
for i in range(100,150):
    list_part.append(df_repeated["Blue-General"][i])
    list_target.append(3)

In [21]:
for i in range(150,200):
    list_part.append(df_repeated["Green-Details"][i])
    list_target.append(4)

In [22]:
for i in range(200,250):
    list_part.append(df_repeated["Orange-Perf"][i])
    list_target.append(5)

In [23]:
for i in range(250,300):
    list_part.append(df_repeated["Borwn-Structure"][i])
    list_target.append(6)

In [24]:
for i in range(300,350):
    list_part.append(df_repeated["Yellow-Conclusion"][i])
    list_target.append(7)

In [25]:
df_repeated["Main_Sentence"] = list_part
df_repeated["Sentence_Type"] = list_target

In [52]:
display(df_repeated.head(10))
display(df_repeated.shape)

,Blog,Publication Date,Reviewer,Title of the Play,Playwright,Theatre,Review with Zoning,Intro-Purple,Red-Plot,Blue-General,Green-Details,Orange-Perf,Borwn-Structure,Yellow-Conclusion,Latitude,Longitude,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Start_Review_1,End_Review_1,Start_Review_2,End_Review_2,Start_Review_3,End_Review_3,Nbr_of_Sentences,Length_Start_Count_Words_1,Length_End_Count_Words_1,Length_Start_Count_Words_2,Length_End_Count_Words_2,Length_Start_Count_Words_3,Length_End_Count_Words_3,Question_Mark_Start_1,Question_Mark_End_1,Question_Mark_Start_2,Question_Mark_End_2,Question_Mark_Start_3,Question_Mark_End_3,Exclamation_Mark_Start_1,Exclamation_Mark_End_1,Exclamation_Mark_Start_2,Exclamation_Mark_End_2,Exclamation_Mark_Start_3,Exclamation_Mark_End_3,Word_Average_Start_1,Word_Average_End_1,Word_Average_Start_2,Word_Average_End_2,Word_Average_Start_3,Word_Average_End_3,Main_Sentence,Sentence_Type
0,DAILY TELEGRAPH,09.03.10,Charles Spencer,SWEET NOTHINGS,Arthur Schnitzler,Young Vic Theatre,Sigmund Freud was a great admirer of the Austr...,Sigmund Freud was a great admirer of the Austr...,"In the first act, we watch as two Viennese po...",NaN,NaN,There are some superb performances. Tom Hughe...,NaN,With strong support from David Sibley as Chris...,51.503566,-0.101745,0,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,2,0,0,0,1,1,0,2,Story_Cat,0.217,0.691,0.092,-0.9787,-0.089583,0.531548,0.15,0.39,0.09,0.24,0.12,"[('falling', 0.1670849527972756), ('cally', 0....",['Frank More just wants to teach history'],"["" Nevertheless, Milner's rabbithole view of a...","['Frank More just wants to teach history', "" B...","["" There are some breathlessly funny sequences...","['Frank More just wants to teach history', "" B...","[' But as events spiral into chaos, a mix of s...",8,7,28,58,43,76,63,0,0,1,0,1,0,0,0,0,0,0,0,4.57,5.93,5.02,5.59,4.76,5.38,Sigmund Freud was a great admirer of the Austr...,1
1,DAILY EXPRESS,10.03.10,Paul Callan,LOVE NEVER DIES,Andrew Lloyd,Adelphi Theatre,Love never dies has suffered a rocky journey t...,Love never dies has suffered a rocky journey t...,"Lloyd Webber, with lyrics by Glen Slater, has ...",NaN,Some of the songs are lush and romantic. ’Til ...,"Similarly the American-born Miss Boggess, maki...",NaN,This is an elegant and clever sequel to Phanto...,51.510052,-0.122881,1,0,0,0,0,0,7,0,0,1,0,0,0,1,2,1,1,0,0,0,0,1,4,1,0,3,0,2,1,2,4,Show_Cat,0.039,0.781,0.180,0.9968,0.182049,0.560692,0.05,0.30,0.18,0.32,0.15,"[('singing', 0.1669848885915849), ('lloyd', 0....",['LOVE NEVER DIES has suffered a rocky journey...,[' It is a great night out'],['LOVE NEVER DIES has suffered a rocky journey...,[' This is an elegant and clever sequel to Pha...,['LOVE NEVER DIES has suffered a rocky journey...,[' And the special effects -they include singi...,25,11,7,30,35,51,60,0,0,0,0,0,0,0,0,0,0,0,0,4.27,3.00,5.00,4.88,4.82,4.81,Love never dies has suffered a rocky journey t...,1
2,SUNDAY EXPRESS,28.03.10,Mark Shenton,THE WHITE GUARD,Andrew Upton,Lyttelton Theatre,The clock is turned back in every case this we...,The clock is turned back in every case this we...,"The sole woman, Elena (a forceful Justine Mitc...",NaN,NaN,NaN,NaN,"Written across a broad, epic canvas and given...",51.507219,-0.114367,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.060,0.809,0.131,0.9052,0.195650,0.377885,0.07,0.55,0.10,0.10,0.17,"[('davies', 0.294100664407846), ('forces', 0.2...",n.a,n.a,n.a,n.a,n.a,n.a,6,1,1,2,2,3,3,0,0,0,0,0,0,0,0,0,0,0,0,1.00,1.00,1.00,1.00,1.00,1.00,The clock is turned back in every case this we...

(350, 93)

In [27]:
df_repeated.to_excel("df_repeated.xlsx")

In [28]:
df_repeated["Intro-Purple"].nunique()

45

In [29]:
df["Intro-Purple"].nunique()

45

In [16]:
df_repeated.to_excel("df_review.xlsx", index=False)

In [30]:
df_repeated.columns

Index(['Blog', 'Publication Date', 'Reviewer', 'Title of the Play',
       'Playwright', 'Theatre  ', 'Review with Zoning', 'Intro-Purple',
       'Red-Plot', 'Blue-General', 'Green-Details', 'Orange-Perf',
       'Borwn-Structure', 'Yellow-Conclusion', 'Latitude', 'Longitude',
       'Women/Feminism', 'LGBT / Queer / Sexuality / Gender', 'Disabled',
       'Politics', 'Supernatural', 'Science', 'Body', 'Childhood',
       'Cultural difference / Race', 'Death', 'Education', 'Family',
       'Environment', 'Friendship', 'Love', 'Identity', 'Memory',
       'Relationships', 'Religion', 'Violence', 'Spectator_Cat',
       'Audience_Cat', 'Show_Cat', 'Story_Cat', 'Decor_Cat', 'Theatre_Cat',
       'Production_Cat', 'Character_Cat', 'End_Cat', 'Beginning_Cat',
       'Max_Value', 'Cluster', 'Neg', 'Neu', 'Pos', 'Compound', 'Polarity',
       'Subjectivity', 'Angry', 'Fear', 'Happy', 'Sad', 'Surpise',
       'Top_10_Keywords', 'Start_Review_1', 'End_Review_1', 'Start_Review_2',
       'End_R

In [45]:
df_repeated.shape

(350, 93)

In [44]:
display(df_repeated[df_repeated["Intro-Purple"].isna() == True].shape)
display(df_repeated[df_repeated["Red-Plot"].isna() == True].shape)
display(df_repeated[df_repeated["Blue-General"].isna() == True].shape)
display(df_repeated[df_repeated["Orange-Perf"].isna() == True].shape)
display(df_repeated[df_repeated["Borwn-Structure"].isna() == True].shape)
display(df_repeated[df_repeated["Yellow-Conclusion"].isna() == True].shape)

(35, 93)

(0, 93)

(126, 93)

(126, 93)

(273, 93)

(42, 93)

In [41]:
df_repeated_filtered[df_repeated_filtered["Main_Sentence"].isna() == True].shape

(0, 93)

In [42]:
df_repeated[df_repeated["Main_Sentence"].isna() == True].shape

(109, 93)

In [54]:
df_repeated.shape

(350, 93)

In [55]:
df_repeated_filtered.shape

(241, 93)

In [58]:
df_repeated_filtered[df_repeated_filtered["Main_Sentence"].isna() == True].shape

(0, 93)

In [57]:
df_repeated_filtered = df_repeated[df_repeated['Main_Sentence'].isna() == False].reset_index(drop=True)
display(df_repeated_filtered.head(3))
display(df_repeated_filtered.shape)

,Blog,Publication Date,Reviewer,Title of the Play,Playwright,Theatre,Review with Zoning,Intro-Purple,Red-Plot,Blue-General,Green-Details,Orange-Perf,Borwn-Structure,Yellow-Conclusion,Latitude,Longitude,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Start_Review_1,End_Review_1,Start_Review_2,End_Review_2,Start_Review_3,End_Review_3,Nbr_of_Sentences,Length_Start_Count_Words_1,Length_End_Count_Words_1,Length_Start_Count_Words_2,Length_End_Count_Words_2,Length_Start_Count_Words_3,Length_End_Count_Words_3,Question_Mark_Start_1,Question_Mark_End_1,Question_Mark_Start_2,Question_Mark_End_2,Question_Mark_Start_3,Question_Mark_End_3,Exclamation_Mark_Start_1,Exclamation_Mark_End_1,Exclamation_Mark_Start_2,Exclamation_Mark_End_2,Exclamation_Mark_Start_3,Exclamation_Mark_End_3,Word_Average_Start_1,Word_Average_End_1,Word_Average_Start_2,Word_Average_End_2,Word_Average_Start_3,Word_Average_End_3,Main_Sentence,Sentence_Type
0,DAILY TELEGRAPH,09.03.10,Charles Spencer,SWEET NOTHINGS,Arthur Schnitzler,Young Vic Theatre,Sigmund Freud was a great admirer of the Austr...,Sigmund Freud was a great admirer of the Austr...,"In the first act, we watch as two Viennese po...",NaN,NaN,There are some superb performances. Tom Hughe...,NaN,With strong support from David Sibley as Chris...,51.503566,-0.101745,0,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,2,0,0,0,1,1,0,2,Story_Cat,0.217,0.691,0.092,-0.9787,-0.089583,0.531548,0.15,0.39,0.09,0.24,0.12,"[('falling', 0.1670849527972756), ('cally', 0....",['Frank More just wants to teach history'],"["" Nevertheless, Milner's rabbithole view of a...","['Frank More just wants to teach history', "" B...","["" There are some breathlessly funny sequences...","['Frank More just wants to teach history', "" B...","[' But as events spiral into chaos, a mix of s...",8,7,28,58,43,76,63,0,0,1,0,1,0,0,0,0,0,0,0,4.57,5.93,5.02,5.59,4.76,5.38,Sigmund Freud was a great admirer of the Austr...,1
1,DAILY EXPRESS,10.03.10,Paul Callan,LOVE NEVER DIES,Andrew Lloyd,Adelphi Theatre,Love never dies has suffered a rocky journey t...,Love never dies has suffered a rocky journey t...,"Lloyd Webber, with lyrics by Glen Slater, has ...",NaN,Some of the songs are lush and romantic. ’Til ...,"Similarly the American-born Miss Boggess, maki...",NaN,This is an elegant and clever sequel to Phanto...,51.510052,-0.122881,1,0,0,0,0,0,7,0,0,1,0,0,0,1,2,1,1,0,0,0,0,1,4,1,0,3,0,2,1,2,4,Show_Cat,0.039,0.781,0.180,0.9968,0.182049,0.560692,0.05,0.30,0.18,0.32,0.15,"[('singing', 0.1669848885915849), ('lloyd', 0....",['LOVE NEVER DIES has suffered a rocky journey...,[' It is a great night out'],['LOVE NEVER DIES has suffered a rocky journey...,[' This is an elegant and clever sequel to Pha...,['LOVE NEVER DIES has suffered a rocky journey...,[' And the special effects -they include singi...,25,11,7,30,35,51,60,0,0,0,0,0,0,0,0,0,0,0,0,4.27,3.00,5.00,4.88,4.82,4.81,Love never dies has suffered a rocky journey t...,1
2,SUNDAY EXPRESS,28.03.10,Mark Shenton,THE WHITE GUARD,Andrew Upton,Lyttelton Theatre,The clock is turned back in every case this we...,The clock is turned back in every case this we...,"The sole woman, Elena (a forceful Justine Mitc...",NaN,NaN,NaN,NaN,"Written across a broad, epic canvas and given...",51.507219,-0.114367,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.060,0.809,0.131,0.9052,0.195650,0.377885,0.07,0.55,0.10,0.10,0.17,"[('davies', 0.294100664407846), ('forces', 0.2...",n.a,n.a,n.a,n.a,n.a,n.a,6,1,1,2,2,3,3,0,0,0,0,0,0,0,0,0,0,0,0,1.00,1.00,1.00,1.00,1.00,1.00,The clock is turned back in every case this we...

(241, 93)

In [59]:
df_sentence = df_repeated_filtered[["Main_Sentence"]]
df_sentence.head(3)

,Main_Sentence
0,Sigmund Freud was a great admirer of the Austr...
1,Love never dies has suffered a rocky journey t...
2,The clock is turned back in every case this we...


In [60]:
df_sentence["Women/Feminism"] = 0
df_sentence["LGBT / Queer / Sexuality / Gender"] = 0
df_sentence["Disabled"] = 0
df_sentence["Politics"] = 0
df_sentence["Supernatural"] = 0
df_sentence["Science"] = 0
df_sentence["Body"] = 0
df_sentence["Childhood"] = 0
df_sentence["Cultural difference / Race"] = 0
df_sentence["Death"] = 0
df_sentence["Education"] = 0
df_sentence["Family"] = 0
df_sentence["Environment"] = 0
df_sentence["Friendship"] = 0
df_sentence["Love"] = 0
df_sentence["Identity"] = 0
df_sentence["Memory"] = 0
df_sentence["Relationships"] = 0
df_sentence["Religion"] = 0
df_sentence["Violence"] = 0

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [61]:
list_feminism = ["womanism","feminism","women","feminist","modernism","daughter","girl","mother","woman"]
list_woman_score = []
for j in range(0,len(df_sentence["Women/Feminism"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Women/Feminism"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [62]:
list_feminism = ["lgbt","queer","sexuality","gender","lesbian","gay","bisexual","transgender","sensuality","sexiness","passion","desire","lust","erotism","feminity","masculinity","womanliness","manliness"]
list_woman_score = []
for j in range(0,len(df_sentence["LGBT / Queer / Sexuality / Gender"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["LGBT / Queer / Sexuality / Gender"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [63]:
list_feminism = ["politics","election","government","diplomatic","tactical","policy","state"]
list_woman_score = []
for j in range(0,len(df_sentence["Politics"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Politics"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [64]:
list_feminism = ["supernatural","celestial","fairy","ghostly","evenly","paranormal","phenomenal","superhuman","metaphysical","phantom","unknown"]
list_woman_score = []
for j in range(0,len(df_sentence["Supernatural"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Supernatural"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [65]:
list_feminism = ["knowledge","wisdom","intelligence","physics","mathemtics","knowledge","erudition","discipline","science"]
list_woman_score = []
for j in range(0,len(df_sentence["Science"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Science"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [66]:
list_feminism = ["body","anatomy","figure","hand","foot","eye","hear","leg","arm","skeleton","bones","shape","corpse"]
list_woman_score = []
for j in range(0,len(df_sentence["Body"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Body"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [67]:
list_feminism = ["childhood","infant","youth","infancy","child","baby","newborn","infant","youngster","toddler"]
list_woman_score = []
for j in range(0,len(df_sentence["Childhood"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Childhood"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [68]:
list_feminism = ["cultural","difference","race","diversity","cross-culturalism","multiculturalism","ethnic"]
list_woman_score = []
for j in range(0,len(df_sentence["Cultural difference / Race"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Cultural difference / Race"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [69]:
list_feminism = ["death","demise","dying","end","murder","expiry","passing","assassination","slaying","slaughter"]
list_woman_score = []
for j in range(0,len(df_sentence["Death"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Death"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [70]:
list_feminism = ["education","teaching","schooling","tutor","teach","coaching","learning","knowledge","scholar"]
list_woman_score = []
for j in range(0,len(df_sentence["Education"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Education"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [71]:
list_feminism = ["household","ménage","kids","father","mother","child","birth","ancestry","clan","tribe","folk","family"]
list_woman_score = []
for j in range(0,len(df_sentence["Family"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Family"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [72]:
list_feminism = ["environment","climate","habitat","surroundings","ambiance","conditions"]
list_woman_score = []
for j in range(0,len(df_sentence["Environment"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Environment"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [73]:
list_feminism = ["friendship","friend","bond","love","intimacy","devotion","coalition","association"]
list_woman_score = []
for j in range(0,len(df_sentence["Friendship"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Friendship"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [74]:
list_feminism = ["love","affection","appreciation","devotion","emotion","fondness","friendship","lust","passion"]
list_woman_score = []
for j in range(0,len(df_sentence["Love"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Love"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [75]:
list_feminism = ["identity","character","existence","identification","integrity","name","personality","status"]
list_woman_score = []
for j in range(0,len(df_sentence["Identity"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Identity"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [76]:
list_feminism = ["memory","consciousness","mind","recollection","remembrance","thought","awareness"]
list_woman_score = []
for j in range(0,len(df_sentence["Memory"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Memory"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [77]:

list_feminism = ["relationship","mariage","link","exchange","liaison","rapport","relation"]
list_woman_score = []
for j in range(0,len(df_sentence["Relationships"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Relationships"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [78]:
list_feminism = ["religion","church","creed","cult","denomination","doctrine","morality","ritual","sect"]
list_woman_score = []
for j in range(0,len(df_sentence["Religion"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Religion"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [79]:
list_feminism = ["assault","attack","brutality","clash","cruelty","fight","ferocity","harsh"]
list_woman_score = []
for j in range(0,len(df_sentence["Violence"])):
    score = 0
    for i in range(0,len(list_feminism)):
        if list_feminism[i] in df_sentence["Main_Sentence"][j]:
            score += 1
    list_woman_score.append(score)
df_sentence["Violence"] = list_woman_score

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [80]:
list_spectator = ["spectator","bystander","fan","moviegoer","observer","onlooker","theatergoer","viewer","beholder","clapper","eyewitness","looker","looker-on","perceiver","playgoer","seer","standee","watcher","witness","gaper","gazer","showgoer","stander-by"]
list_audience = ["audience","spectators","company","house","crowd","gathering","gallery","assembly","viewers","listeners","patrons","congregation","turnout","onlookers","throng","assemblage"]
list_show = ["show","entertainment","performance","play","production","drama","musical","presentation","theatrical performance","exhibition","fair","display","parade","expo"]
list_story = ["story","tale","romance","narrative","record","history","version","novel","legend","chronicle","yarn","recital","narration"]
list_decor = ["color scheme","decoration","interior design","adornment","ornamentation"]
list_theatre = ["theatre","hall","room","auditorium","theater","troupe","drama","setting","site","scene","theatrics","coliseum","amphitheatre"]
list_production = ["production","creation","development","fashioning","composition","origination","rendering"]
list_character = ["character","person","individual","type","guy","fellow","role","persona","nature","spirirt","emotions","habit"]
list_end = ["end","close","ending","finish","expiry","expiration","conclusion","ending","climax","completion","finale","culmination","denouement","consummation","finish","closure"]
list_beginning = ["beginning","start","opening","chance","source","opportunity","birth","origin","introduction","outset","starting point","onset","overture","initiation","inauguration","inception","commencement","opening move"]

In [81]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_spectator:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["Spectator_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [82]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_audience:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["Audience_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [83]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_show:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["Show_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [84]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_story:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["Story_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [85]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_decor:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["Decor_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [86]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_theatre:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["Theatre_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [87]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_production:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["Production_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [88]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_character:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["Character_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [89]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_end:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["End_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [90]:
list_counter=[]
for i in range(0,len(df_sentence["Main_Sentence"])):
    c = 0
    for j in list_beginning:
        if type(df_sentence["Main_Sentence"][i]) != float:
            if j in df_sentence["Main_Sentence"][i].lower():
                c= c+1
    list_counter.append(c)
df_sentence["Beginning_Cat"] = list_counter

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [91]:
df_sentence2 = df_sentence.copy()
df_sentence2.head(3)

,Main_Sentence,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0


In [92]:
df_key = df_sentence2[["Spectator_Cat","Audience_Cat","Show_Cat","Story_Cat","Decor_Cat","Theatre_Cat","Production_Cat","Character_Cat","End_Cat","Beginning_Cat"]]

In [93]:
df_key['max_value'] = df_key.max(axis=1)

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [94]:
list_max = []
for i in range(0,len(df_key["Spectator_Cat"])):
    list_max.append(df_key.loc[i].idxmax(axis=1))

In [95]:
df_sentence2["Max_Value"] = df_key['max_value']
df_sentence2["Cluster"] = list_max

In [100]:
sid = SentimentIntensityAnalyzer()

In [65]:
df_sentence2.head(3)

,Main_Sentence,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat


In [97]:
df_sentence2.shape

(241, 33)

In [101]:
df_sentence2 = df_sentence2.copy()
list_neg = []
list_neu = []
list_pos = []
list_compound = []
list_polarity = []
list_subjectivity = []
for i in range(0,len(df_sentence2["Main_Sentence"])):
    score1 = sid.polarity_scores(df_sentence2["Main_Sentence"][i])
    score2 = TextBlob(df_sentence2["Main_Sentence"][i]).sentiment
    list_neg.append(score1['neg'])
    list_neu.append(score1['neu'])
    list_pos.append(score1['pos'])
    list_compound.append(score1['compound'])
    list_polarity.append(score2[0])
    list_subjectivity.append(score2[1])

In [102]:
df_sentence2["Neg"] = list_neg
df_sentence2["Neu"] = list_neu
df_sentence2["Pos"] = list_pos
df_sentence2["Compound"] = list_compound
df_sentence2["Polarity"] = list_polarity
df_sentence2["Subjectivity"] = list_subjectivity

In [103]:
display(df_sentence2.head(3))
display(df_sentence2.shape)

,Main_Sentence,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333


(241, 39)

In [104]:
df_sentence2 = df_sentence2
df_sentence2["Emotion"] = ""
df_sentence2["Emotion"][0:5] = df_sentence2["Main_Sentence"][0:5].apply(lambda x:te.get_emotion(x))

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [105]:
list_emotion_angry = []
list_emotion_fear = []
list_emotion_happy = []
list_emotion_sad = []
list_emotion_surprise = []
for i in range(0,len(df_sentence2["Main_Sentence"])):
  print(i)
  emotion =te.get_emotion(df_sentence2["Main_Sentence"][i])
  list_emotion_angry.append(emotion["Angry"])
  list_emotion_fear.append(emotion["Fear"])
  list_emotion_happy.append(emotion["Happy"])
  list_emotion_sad.append(emotion["Sad"])
  list_emotion_surprise.append(emotion["Surprise"])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240


In [106]:
df_sentence2["Angry"] = list_emotion_angry
df_sentence2["Fear"] = list_emotion_fear
df_sentence2["Happy"] = list_emotion_happy
df_sentence2["Sad"] = list_emotion_sad
df_sentence2["Surpise"] = list_emotion_surprise

In [107]:
df_sentence2 = df_sentence2

In [108]:
list_keywords = []
for i in range(0,len(df_sentence2['Main_Sentence'])):
    print(i)
    text_en = (df_sentence2['Main_Sentence'][i])
    list_keywords.append(keywords(text_en,words = 10,scores = True, lemmatize = True))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240


In [109]:
df_sentence2['Top_10_Keywords'] = list_keywords

In [110]:
df_sentence2.head(4)

,Main_Sentence,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Emotion,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[(sexuality, 0.2538867748491288), (harrower, 0..."
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[(otto, 0.3578513897480544), (lloyd, 0.3578513..."
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[(theatrical, 0.28284610378675523), (women, 0...."
3,"Written across a broad, epic canvas and given...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,Show_Cat,0.000,1.00,0.000,0.0000,0.08125,0.356250,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ...",0.00,1.00,0.00,0.00,0.00,"[(davies, 0.41061373901915704), (epic canvas, ..."


In [111]:
Length_Start_Count_Words = []


for j in range(0,len(df_sentence2["Main_Sentence"])):
    if len(df_sentence2["Main_Sentence"][j]) > 0:
        Length_Start_Count_Words.append(len(df_sentence2["Main_Sentence"][j]))
    else:
        Length_Start_Count_Words.append("n.a")

In [112]:
df_sentence2["Count_Words"] = Length_Start_Count_Words

In [113]:
question_mark_start_1 = []
for i in range(0,len(df_sentence2["Main_Sentence"])):
    if "?" in df_sentence2["Main_Sentence"][i]:
        question_mark_start_1.append(1)
    else:
        question_mark_start_1.append(0)

In [114]:
df_sentence2["Question_Mark"] = question_mark_start_1

In [115]:
exclamation_mark_start_1 = []
for i in range(0,len(df_sentence2["Main_Sentence"])):
    if "!" in df_sentence2["Main_Sentence"][i]:
        exclamation_mark_start_1.append(1)
    else:
        exclamation_mark_start_1.append(0)

In [116]:
df_sentence2["Exclamation_Mark"] = exclamation_mark_start_1

In [117]:
word_average_start_1 = []
for i in range(0,len(df_sentence2["Main_Sentence"])):
    if df_sentence2["Main_Sentence"][i] != '':
        words = df_sentence2["Main_Sentence"][i].split()
        if len(words) > 0:
            average = sum(len(word) for word in words) / len(words)
            avg = round(average,2)
            word_average_start_1.append(avg)
        else:
            word_average_start_1.append('n.a')
    else:
        word_average_start_1.append('n.a')

In [118]:
df_sentence2["Word_Average"] = word_average_start_1

In [119]:
df_sentence2.head(3)

,Main_Sentence,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Emotion,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Count_Words,Question_Mark,Exclamation_Mark,Word_Average
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[(sexuality, 0.2538867748491288), (harrower, 0...",761,0,0,5.46
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[(otto, 0.3578513897480544), (lloyd, 0.3578513...",429,0,0,4.89
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[(theatrical, 0.28284610378675523), (women, 0....",481,0,0,4.95


In [126]:
df_repeated_filtered.head(3)
df_repeated_filtered.shape

(241, 93)

In [127]:
df_repeated_filtered2 = df_repeated_filtered[["Main_Sentence","Sentence_Type"]]
df_repeated_filtered2.head(3)

,Main_Sentence,Sentence_Type
0,Sigmund Freud was a great admirer of the Austr...,1
1,Love never dies has suffered a rocky journey t...,1
2,The clock is turned back in every case this we...,1


In [128]:
display(df_repeated_filtered2.head(3))
display(df_repeated_filtered2.shape)
display(df_sentence2.head(3))
display(df_sentence2.shape)

,Main_Sentence,Sentence_Type
0,Sigmund Freud was a great admirer of the Austr...,1
1,Love never dies has suffered a rocky journey t...,1
2,The clock is turned back in every case this we...,1


(241, 2)

,Main_Sentence,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Emotion,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Count_Words,Question_Mark,Exclamation_Mark,Word_Average
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[(sexuality, 0.2538867748491288), (harrower, 0...",761,0,0,5.46
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[(otto, 0.3578513897480544), (lloyd, 0.3578513...",429,0,0,4.89
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[(theatrical, 0.28284610378675523), (women, 0....",481,0,0,4.95


(241, 50)

In [123]:
df_sentence3 = pd.merge(df_sentence2,df_repeated_filtered2, left_index=True, right_index=True,how="left")
display(df_sentence3.head(3))
display(df_sentence3.shape)

,Main_Sentence_x,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Emotion,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Count_Words,Question_Mark,Exclamation_Mark,Word_Average,Main_Sentence_y,Sentence_Type
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[(sexuality, 0.2538867748491288), (harrower, 0...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[(otto, 0.3578513897480544), (lloyd, 0.3578513...",429,0,0,4.89,Love never dies has suffered a rocky journey t...,1
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[(theatrical, 0.28284610378675523), (women, 0....",481,0,0,4.95,The clock is turned back in every case this we...,1


(241, 52)

In [124]:
df_sentence3.head(3)
df_sentence3.shape

(241, 52)

In [130]:
list_I = []
for i in range(0,len(df_sentence3["Main_Sentence_x"])):
    list_I.append(df_sentence3["Main_Sentence_x"][i].lower().split(" ").count("I"))

In [131]:
len(list_I)

241

In [132]:
list_we = []
for i in range(0,len(df_sentence3["Main_Sentence_x"])):
    list_we.append(df_sentence3["Main_Sentence_x"][i].lower().split(" ").count("we"))

In [133]:
len(list_we)

241

In [134]:
list_you = []
for i in range(0,len(df_sentence3["Main_Sentence_x"])):
    list_you.append(df_sentence3["Main_Sentence_x"][i].lower().split(" ").count("you"))

In [135]:
len(list_you)

241

In [136]:
list_she = []
for i in range(0,len(df_sentence3["Main_Sentence_x"])):
    list_she.append(df_sentence3["Main_Sentence_x"][i].lower().split(" ").count("she"))

In [137]:
len(list_she)

241

In [145]:
list_he = []
for i in range(0,len(df_sentence3["Main_Sentence_x"])):
    list_he.append(df_sentence3["Main_Sentence_x"][i].lower().split(" ").count("he"))

In [146]:
len(list_he)

241

In [147]:
list_it = []
for i in range(0,len(df_sentence3["Main_Sentence_x"])):
    list_it.append(df_sentence3["Main_Sentence_x"][i].lower().split(" ").count("it"))

In [148]:
len(list_it)

241

In [149]:
df_sentence3["Pronoun_I"] = list_I
df_sentence3["Pronoun_We"] = list_we
df_sentence3["Pronoun_You"] = list_you
df_sentence3["Pronoun_She"] = list_she
df_sentence3["Pronoun_He"] = list_he
df_sentence3["Pronoun_It"] = list_it

In [150]:
df_sentence3.head(3)

,Main_Sentence_x,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Emotion,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Count_Words,Question_Mark,Exclamation_Mark,Word_Average,Main_Sentence_y,Sentence_Type,Pronoun_I,Pronoun_We,Pronoun_You,Pronoun_She,Pronoun_He,Pronoun_It
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.80,0.170,0.9531,0.20625,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[(sexuality, 0.2538867748491288), (harrower, 0...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,0,0,1,0,0,2
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.72,0.141,0.0330,0.06250,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[(otto, 0.3578513897480544), (lloyd, 0.3578513...",429,0,0,4.89,Love never dies has suffered a rocky journey t...,1,0,0,0,0,0,0
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.70,0.196,0.8462,0.26455,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[(theatrical, 0.28284610378675523), (women, 0....",481,0,0,4.95,The clock is turned back in every case this we...,1,0,0,0,0,0,1


In [135]:
df_sentence3.shape

(241, 54)

In [129]:
import spacy
sp = spacy.load('en_core_web_sm')

In [130]:
Type_word_k =[]
Type_word_v =[]
for i in range(0,len(df_sentence3["Main_Sentence_x"])):
    sen = sp(df_sentence3["Main_Sentence_x"][i])
    num_pos = sen.count_by(spacy.attrs.POS)
    list_k = []
    list_v = []
    for k,v in sorted(num_pos.items()):
        list_k.append(sen.vocab[k].text)
        list_v.append(v)
    Type_word_k.append(list_k)
    Type_word_v.append(list_v)

In [131]:
df_sentence3["Type_word_k"] = Type_word_k
df_sentence3["Type_word_v"] = Type_word_v

In [132]:
df_sentence3.head(20)

,Main_Sentence_x,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Emotion,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Count_Words,Question_Mark,Exclamation_Mark,Word_Average,Main_Sentence_y,Sentence_Type,Type_word_k,Type_word_v
0,Sigmund Freud was a great admirer of the Austr...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,Show_Cat,0.031,0.800,0.170,0.9531,0.206250,0.716667,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",0.06,0.18,0.06,0.41,0.29,"[(sexuality, 0.2538867748491288), (harrower, 0...",761,0,0,5.46,Sigmund Freud was a great admirer of the Austr...,1,"[ADJ, ADP, ADV, AUX, CCONJ, DET, NOUN, NUM, PA...","[15, 19, 4, 6, 1, 13, 22, 3, 2, 6, 16, 20, 3, ..."
1,Love never dies has suffered a rocky journey t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.139,0.720,0.141,0.0330,0.062500,0.583333,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14...",0.00,0.14,0.14,0.57,0.14,"[(otto, 0.3578513897480544), (lloyd, 0.3578513...",429,0,0,4.89,Love never dies has suffered a rocky journey t...,1,"[ADJ, ADP, ADV, AUX, CCONJ, DET, NOUN, PART, P...","[4, 7, 5, 9, 2, 11, 12, 1, 4, 8, 13, 2, 9]"
2,The clock is turned back in every case this we...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.104,0.700,0.196,0.8462,0.264550,0.433333,"{'Happy': 0.2, 'Angry': 0.13, 'Surprise': 0.27...",0.13,0.40,0.20,0.00,0.27,"[(theatrical, 0.28284610378675523), (women, 0....",481,0,0,4.95,The clock is turned back in every case this we...,1,"[ADJ, ADP, ADV, AUX, CCONJ, DET, NOUN, NUM, PA...","[9, 12, 3, 4, 4, 15, 16, 1, 1, 3, 8, 6, 1, 7]"
3,"Written across a broad, epic canvas and given...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,Show_Cat,0.000,1.000,0.000,0.0000,0.081250,0.356250,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ...",0.00,1.00,0.00,0.00,0.00,"[(davies, 0.41061373901915704), (epic canvas, ...",175,0,0,4.83,"Written across a broad, epic canvas and given...",1,"[ADJ, ADP, CCONJ, DET, NOUN, NUM, PUNCT, VERB,...","[3, 5, 2, 5, 10, 1, 3, 4, 1]"
4,How do you feel about porn? Is it empowering o...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Show_Cat,0.075,0.780,0.145,0.5022,0.250000,0.350000,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.33,...",0.00,0.50,0.00,0.17,0.33,"[(musical answers, 0.3696951272282225), (extre...",165,1,0,5.15,How do you feel about porn? Is it empowering o...,1,"[ADJ, ADP, ADV, AUX, CCONJ, DET, NOUN, PRON, P...","[2, 2, 3, 3, 2, 4, 4, 2, 2, 6, 3]"
5,"On paper, Counted? sounds a little dull. Bille...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2,0,0,1,1,0,0,0,2,Show_Cat,0.058,0.822,0.120,0.6232,-0.002034,0.410119,,0.10,0.50,0.00,0.40,0.00,"[(dull, 0.3570424717754877), (theatre, 0.35704...",484,1,0,5.37,"On paper, Counted? sounds a little dull. Bille...",1,"[ADJ, ADP, ADV, AUX, CCONJ, DET, NOUN, NUM, PA...","[7, 7, 6, 3, 3, 9, 17, 1, 5, 4, 2, 11, 3, 13]"
6,"A sinister story of music, madness and repulsi...",0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,2,0,0,1,0,1,0,2,Story_Cat,0.228,0.694,0.078,-0.9477,0.088462,0.715934,,0.16,0.37,0.11,0.26,0.11,"[(harris, 0.2218028377600289), (hypnotic, 0.22...",570,0,0,5.21,"A sinister story of music, madness and repulsi...",1,"[ADJ, ADP, ADV, AUX, CCONJ, DET, NOUN, NUM, PA...","[12, 15, 4, 1, 5, 12, 23, 1, 2, 1, 7, 14, 1, 10]"
7,Other actors play multiple roles in multiple w...,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,Show_Cat,0.122,0.792,0.086,-0.2484,0.010417,0.177083,,0.00,0.50,0.00,0.50,0.00,"[(play multiple, 0.33977114298975175), (glum, ...",163,0,0,5.31,Other actors play multiple roles in multiple w.

In [133]:
df_sentence3.to_excel("zoning_v2.xlsx")

In [134]:
df_sentence3.to_excel("zoning_v2.xlsx")

In [194]:
df.head(20)

,Blog,Publication Date,Reviewer,Title of the Play,Playwright,Theatre,Review with Zoning,Intro-Purple,Red-Plot,Blue-General,Green-Details,Orange-Perf,Borwn-Structure,Yellow-Conclusion,Latitude,Longitude,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Cluster,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Angry,Fear,Happy,Sad,Surpise,Top_10_Keywords,Start_Review_1,End_Review_1,Start_Review_2,End_Review_2,Start_Review_3,End_Review_3,Nbr_of_Sentences,Length_Start_Count_Words_1,Length_End_Count_Words_1,Length_Start_Count_Words_2,Length_End_Count_Words_2,Length_Start_Count_Words_3,Length_End_Count_Words_3,Question_Mark_Start_1,Question_Mark_End_1,Question_Mark_Start_2,Question_Mark_End_2,Question_Mark_Start_3,Question_Mark_End_3,Exclamation_Mark_Start_1,Exclamation_Mark_End_1,Exclamation_Mark_Start_2,Exclamation_Mark_End_2,Exclamation_Mark_Start_3,Exclamation_Mark_End_3,Word_Average_Start_1,Word_Average_End_1,Word_Average_Start_2,Word_Average_End_2,Word_Average_Start_3,Word_Average_End_3
0,DAILY TELEGRAPH,09.03.10,Charles Spencer,SWEET NOTHINGS,Arthur Schnitzler,Young Vic Theatre,Sigmund Freud was a great admirer of the Austr...,Sigmund Freud was a great admirer of the Austr...,"In the first act, we watch as two Viennese po...",NaN,NaN,There are some superb performances. Tom Hughe...,NaN,With strong support from David Sibley as Chris...,51.503566,-0.101745,0,1,0,1,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,2,0,0,0,1,1,0,2,Story_Cat,0.217,0.691,0.092,-0.9787,-0.089583,0.531548,0.15,0.39,0.09,0.24,0.12,"[('falling', 0.1670849527972756), ('cally', 0....",['Frank More just wants to teach history'],"["" Nevertheless, Milner's rabbithole view of a...","['Frank More just wants to teach history', "" B...","["" There are some breathlessly funny sequences...","['Frank More just wants to teach history', "" B...","[' But as events spiral into chaos, a mix of s...",8,7,28,58,43,76,63,0,0,1,0,1,0,0,0,0,0,0,0,4.57,5.93,5.02,5.59,4.76,5.38
1,DAILY EXPRESS,10.03.10,Paul Callan,LOVE NEVER DIES,Andrew Lloyd,Adelphi Theatre,Love never dies has suffered a rocky journey t...,Love never dies has suffered a rocky journey t...,"Lloyd Webber, with lyrics by Glen Slater, has ...",NaN,Some of the songs are lush and romantic. ’Til ...,"Similarly the American-born Miss Boggess, maki...",NaN,This is an elegant and clever sequel to Phanto...,51.510052,-0.122881,1,0,0,0,0,0,7,0,0,1,0,0,0,1,2,1,1,0,0,0,0,1,4,1,0,3,0,2,1,2,4,Show_Cat,0.039,0.781,0.180,0.9968,0.182049,0.560692,0.05,0.30,0.18,0.32,0.15,"[('singing', 0.1669848885915849), ('lloyd', 0....",['LOVE NEVER DIES has suffered a rocky journey...,[' It is a great night out'],['LOVE NEVER DIES has suffered a rocky journey...,[' This is an elegant and clever sequel to Pha...,['LOVE NEVER DIES has suffered a rocky journey...,[' And the special effects -they include singi...,25,11,7,30,35,51,60,0,0,0,0,0,0,0,0,0,0,0,0,4.27,3.00,5.00,4.88,4.82,4.81
2,SUNDAY EXPRESS,28.03.10,Mark Shenton,THE WHITE GUARD,Andrew Upton,Lyttelton Theatre,The clock is turned back in every case this we...,The clock is turned back in every case this we...,"The sole woman, Elena (a forceful Justine Mitc...",NaN,NaN,NaN,NaN,"Written across a broad, epic canvas and given...",51.507219,-0.114367,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,Show_Cat,0.060,0.809,0.131,0.9052,0.195650,0.377885,0.07,0.55,0.10,0.10,0.17,"[('davies', 0.294100664407846), ('forces', 0.2...",n.a,n.a,n.a,n.a,n.a,n.a,6,1,1,2,2,3,3,0,0,0,0,0,0,0,0,0,0,0,0,1.00,1.00,1.00,1.00,1.00,1.00
3,THE TIMES,26.03.10,Sam Marlowe,GAMBLING,Tom Holloway,Soho Theatre,Garish lights wink in fetid semidarkness rent ...,"Written across a broad, epic canvas and given...",A woman sele

In [171]:
list_k

['ADP', 'ADV', 'DET', 'NOUN', 'PART', 'PRON', 'PROPN', 'PUNCT', 'VERB']

In [166]:
list_v

[1, 1, 1, 1, 1, 3, 1, 1, 3]

In [158]:
for k,v in sorted(num_pos.items()):
    print(f'{k}. {sen.vocab[k].text:{8}}: {v}')

85. ADP     : 1
86. ADV     : 1
90. DET     : 1
92. NOUN    : 1
94. PART    : 1
95. PRON    : 3
96. PROPN   : 1
97. PUNCT   : 1
100. VERB    : 3


In [89]:
df_sentence3.columns

Index(['Main_Sentence_x', 'Women/Feminism',
       'LGBT / Queer / Sexuality / Gender', 'Disabled', 'Politics',
       'Supernatural', 'Science', 'Body', 'Childhood',
       'Cultural difference / Race', 'Death', 'Education', 'Family',
       'Environment', 'Friendship', 'Love', 'Identity', 'Memory',
       'Relationships', 'Religion', 'Violence', 'Spectator_Cat',
       'Audience_Cat', 'Show_Cat', 'Story_Cat', 'Decor_Cat', 'Theatre_Cat',
       'Production_Cat', 'Character_Cat', 'End_Cat', 'Beginning_Cat',
       'Max_Value', 'Cluster', 'Neg', 'Neu', 'Pos', 'Compound', 'Polarity',
       'Subjectivity', 'Emotion', 'Angry', 'Fear', 'Happy', 'Sad', 'Surpise',
       'Top_10_Keywords', 'Count_Words', 'Question_Mark', 'Exclamation_Mark',
       'Word_Average', 'Main_Sentence_y', 'Sentence_Type'],
      dtype='object')

In [90]:
#df_sentence3["Main_Sentence_x"] = pd.to_numeric(df_sentence3["Main_Sentence_x"])

df_sentence3 = df_sentence3[['Women/Feminism',
       'LGBT / Queer / Sexuality / Gender', 'Disabled', 'Politics',
       'Supernatural', 'Science', 'Body', 'Childhood',
       'Cultural difference / Race', 'Death', 'Education', 'Family',
       'Environment', 'Friendship', 'Love', 'Identity', 'Memory',
       'Relationships', 'Religion', 'Violence', 'Spectator_Cat',
       'Audience_Cat', 'Show_Cat', 'Story_Cat', 'Decor_Cat', 'Theatre_Cat',
       'Production_Cat', 'Character_Cat', 'End_Cat', 'Beginning_Cat',
       'Max_Value', 'Neg', 'Neu', 'Pos', 'Compound', 'Polarity',
       'Subjectivity', 'Angry', 'Fear', 'Happy', 'Sad', 'Surpise',
       'Count_Words', 'Question_Mark', 'Exclamation_Mark',
       'Word_Average', 'Sentence_Type']]

In [91]:
#f_sentence3["Cluster"] = df_sentence3["Cluster"].astype('int64')

In [92]:
df_sentence3.head(3)

,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Angry,Fear,Happy,Sad,Surpise,Count_Words,Question_Mark,Exclamation_Mark,Word_Average,Sentence_Type
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,3,0.031,0.80,0.170,0.9531,0.20625,0.716667,0.06,0.18,0.06,0.41,0.29,761,0,0,5.46,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.139,0.72,0.141,0.0330,0.06250,0.583333,0.00,0.14,0.14,0.57,0.14,429,0,0,4.89,1
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,2,0.104,0.70,0.196,0.8462,0.26455,0.433333,0.13,0.40,0.20,0.00,0.27,481,0,0,4.95,1


In [93]:
X = df_sentence3[[ 'Women/Feminism',
       'LGBT / Queer / Sexuality / Gender', 'Disabled', 'Politics',
       'Supernatural', 'Science', 'Body', 'Childhood',
       'Cultural difference / Race', 'Death', 'Education', 'Family',
       'Environment', 'Friendship', 'Love', 'Identity', 'Memory',
       'Relationships', 'Religion', 'Violence', 'Spectator_Cat',
       'Audience_Cat', 'Show_Cat', 'Story_Cat', 'Decor_Cat', 'Theatre_Cat',
       'Production_Cat', 'Character_Cat', 'End_Cat', 'Beginning_Cat',
       'Max_Value', 'Neg', 'Neu', 'Pos', 'Compound', 'Polarity',
       'Subjectivity', 'Angry', 'Fear', 'Happy', 'Sad', 'Surpise', 'Count_Words', 'Question_Mark', 'Exclamation_Mark',
       'Word_Average']]

In [94]:
y = df_sentence3[["Sentence_Type"]]

## IV - Building Model <a class="anchor" id="4-bullet"></a>

In [95]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [96]:
X_train.head(3)

,Women/Feminism,LGBT / Queer / Sexuality / Gender,Disabled,Politics,Supernatural,Science,Body,Childhood,Cultural difference / Race,Death,Education,Family,Environment,Friendship,Love,Identity,Memory,Relationships,Religion,Violence,Spectator_Cat,Audience_Cat,Show_Cat,Story_Cat,Decor_Cat,Theatre_Cat,Production_Cat,Character_Cat,End_Cat,Beginning_Cat,Max_Value,Neg,Neu,Pos,Compound,Polarity,Subjectivity,Angry,Fear,Happy,Sad,Surpise,Count_Words,Question_Mark,Exclamation_Mark,Word_Average
95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.348,0.515,0.137,-0.5994,-0.700000,0.200000,0.00,0.50,0.0,0.50,0.00,98,0,0,5.19
59,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,0.204,0.719,0.077,-0.6486,0.066667,0.300000,0.00,0.56,0.0,0.33,0.11,190,1,0,4.46
102,0,0,0,0,0,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.208,0.627,0.165,-0.7828,-0.298611,0.448611,0.13,0.33,0.2,0.20,0.13,446,0,0,5.21


In [97]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

## V - Model Predictions <a class="anchor" id="5-bullet"></a>

In [243]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [254]:
labels = ["Purple - Introduction","Red - Plot","Blue - General","Green - Details","Orange - Performances","Brown - Structure","Yellow - Conclusion"]

In [256]:
# Model performance on training set
y_pred_train =clf.predict(X_train)


accuracy = metrics.accuracy_score(y_train, y_pred_train)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(y_train,y_pred_train)
print('Confusion Matrix: \n', cm)
print(classification_report(y_train, y_pred_train, target_names=labels))

Accuracy: 0.96
Confusion Matrix: 
 [[28  0  0  0  0  0  0]
 [ 1 36  0  0  0  0  0]
 [ 0  0 23  0  0  0  0]
 [ 0  0  0 25  0  0  0]
 [ 1  1  0  0 16  0  0]
 [ 0  0  0  1  0  8  0]
 [ 0  0  0  1  1  0 26]]
                       precision    recall  f1-score   support

Purple - Introduction       0.93      1.00      0.97        28
           Red - Plot       0.97      0.97      0.97        37
       Blue - General       1.00      1.00      1.00        23
      Green - Details       0.93      1.00      0.96        25
Orange - Performances       0.94      0.89      0.91        18
    Brown - Structure       1.00      0.89      0.94         9
  Yellow - Conclusion       1.00      0.93      0.96        28

             accuracy                           0.96       168
            macro avg       0.97      0.95      0.96       168
         weighted avg       0.97      0.96      0.96       168



In [257]:
# Predicting the test results
y_pred=clf.predict(X_test)

# Classification results on test set
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}".format(accuracy))

from sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(y_test,y_pred)
print('Confusion Matrix: \n', cm)
print(classification_report(y_test, y_pred, target_names=labels))

Accuracy: 0.14
Confusion Matrix: 
 [[0 3 3 6 3 0 2]
 [2 3 3 2 2 0 1]
 [0 6 1 1 1 0 0]
 [0 0 0 1 0 0 1]
 [0 3 2 5 0 1 3]
 [0 0 1 1 0 0 0]
 [1 4 2 2 1 1 5]]
                       precision    recall  f1-score   support

Purple - Introduction       0.00      0.00      0.00        17
           Red - Plot       0.16      0.23      0.19        13
       Blue - General       0.08      0.11      0.10         9
      Green - Details       0.06      0.50      0.10         2
Orange - Performances       0.00      0.00      0.00        14
    Brown - Structure       0.00      0.00      0.00         2
  Yellow - Conclusion       0.42      0.31      0.36        16

             accuracy                           0.14        73
            macro avg       0.10      0.16      0.11        73
         weighted avg       0.13      0.14      0.13        73



### Great Job !!!